In [1]:
!cat .env

STAGE=PRODUCTION


In [16]:
company = Company.objects.get(name='Kapa Município')
company

<Company: caba5ee3-9e28-4e50-98b2-87c1dae57d82: Kapa Município>

In [17]:
ApprovalFlow.objects.get(company=company, target_model="daily_reports.MultipleDailyReport")

DoesNotExist: ApprovalFlow matching query does not exist.

In [18]:
for permission in UserPermission.objects.filter(companies=company).exclude(name='Inativo'):
    
    
    try:
        if 'MultipleDailyReport' in permission.permissions.keys():
            permission.permissions['MultipleDailyReport']['can_approve'] = True
            print(permission)
        else:
            permission.permissions['multiple_daily_report']['can_approve'] = True
            print(permission)
    except Exception as e:
#         print(f'Não possui parametrização - {permission}')
        pass

    permission.save()

['Kapa Município'] - Suporte
['Kapa Município'] - Encarregado/Apontador
['Kapa Município'] - Administrador do Sistema
['Kapa Município'] - Gestor de Obra
['Kapa Município'] - Diretoria/Gerência
['Kapa Município'] - Apontador Líder
['Kapa Município'] - Coordenador/Medição cliente
['Kapa Município'] - Fiscalização/Cliente


In [19]:
flow = ApprovalFlow(
    name="Fluxo RDO",
    target_model="daily_reports.MultipleDailyReport",
    company=company
)
flow.save()

In [20]:
elaboracao = ApprovalStep(
    name="RDO Criado",
    approval_flow=flow
)
elaboracao.responsible_created_by = True
elaboracao.save()
elaboracao.responsible_users.add(User.objects.get(username='rlcs'))
# elaboracao.responsible_firms.add(Firm.objects.get(name='Programação e Controle', company=company))
# elaboracao.responsible_firms.add(Firm.objects.get(name='Suporte', company=company))

In [21]:
homolog_normal = ApprovalStep(
    name="RDO Aguardando Aprovação",
    approval_flow=flow,
)
homolog_normal.save()
homolog_normal.responsible_users.add(User.objects.get(username='rlcs'))
homolog_normal.responsible_firms.add(Firm.objects.get(name='Aprovadores RDO', company=company))
# homolog_normal.responsible_firms.add(Firm.objects.get(name='Suporte', company=company))

In [22]:
deferido = ApprovalStep(
    name="RDO Aprovado",
    approval_flow=flow,
)
deferido.save()

In [23]:
trans_homolog_normal = ApprovalTransition(
    name="Solicitar aprovação",
    origin=elaboracao,
    destination=homolog_normal,
    condition={
        "===": [{
            "var": "request.action"
            }, "Solicitar aprovação"]
    },
    button={"icon": "assignment", "color": "primary", "variant": "raised"}
)
trans_homolog_normal.save()

In [24]:
trans_homologar_normal = ApprovalTransition(
    name="Aprovar",
    origin=homolog_normal,
    destination=deferido,
    condition={
        "===": [{
            "var": "request.action"
        }, "Aprovar"] 
    },
    button={"icon": "check", "color": "primary", "variant": "raised"},
    callback={"change_fields": [{"name": "editable", "value": False}]}
)
trans_homologar_normal.save()

In [25]:
elaboracao=ApprovalStep.objects.get(approval_flow__company=company, name='RDO Criado')
elaboracao

<ApprovalStep: [Kapa Município]: RDO Criado>

In [26]:
MultipleDailyReport.objects.filter(company=company).update(approval_step=elaboracao)

3

In [27]:
str(deferido.uuid)

'f34635ec-57ad-4453-aad9-4a074d17db2f'

In [28]:
company.metadata['approved_approval_steps'].append(str(deferido.uuid))

In [29]:
company.save()

In [33]:
homolog_normal.responsible_firms.add(*Firm.objects.filter(company=company,name='Aprovadores de RDO'))

In [34]:
homolog_normal.responsible_firms.all()

<QuerySet [<Firm: [b8450161-14d1-4de8-b254-7d1ddc4a9c20: Arteris Intervias] 9234a29c-1edf-4b08-8e01-913d54dd61b5: Aprovadores de RDO>]>